In [1]:
import pandas
from deezy_match_data_construction import construct_deezymatch_data, generate_test_data
from DeezyMatch import train as dm_train
from DeezyMatch import inference as dm_inference
from DeezyMatch import combine_vecs
from DeezyMatch import plot_log
from DeezyMatch import candidate_ranker
import os
import pickle
from fuzzy_string_matching import run as fuzzy_string_matching
from result import result as compute_result

/Users/sarah_shoilee/opt/anaconda3/envs/py39deezy/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


TODO: How did we created gold-standard

TODO: write pseudo code for the steps in k_fold_validation

Number of entity in gold standard:

for i in range(5): # considering k=5
    - take 4 fold data for training and 1 for for testing

In [3]:
def k_fold_validation(file="pm_data/ccrdfconst/wikidata_human_name.pkl", k=5):
    master_df = pandas.read_pickle(file) # load 6178 entities

    DataFrameDict = {i: pandas.DataFrame() for i in range(0,k)} # key = [0, 9], value dataframe chunks
    resultDict = {i: {} for i in range(0,k)}
    
    try:
        offset = round(len(master_df)/k)
        for i in range(k):
            DataFrameDict[i] = master_df[offset*i:offset*(i+1)]
        
        # TODO: run it for [0, 1, 2]
        #for i in range(k):
        for i in [0, 1, 2]:
            train_sample = pandas.DataFrame()
            test_sample = DataFrameDict[i]
            for dict_index in range(0, k):
                if dict_index == i:
                    continue
                train_sample = pandas.concat([train_sample, DataFrameDict[dict_index]], ignore_index=True)

            # GENERATE TRAINING DATA
            train_sample.to_pickle('k_fold_validation/training_sample.pkl')
            construct_deezymatch_data('k_fold_validation/training_sample.pkl',  directory="k_fold_validation/")

            # GENERATE TEST DATA
            test_sample.to_pickle('k_fold_validation/test_sample.pkl')
            generate_test_data('k_fold_validation/test_sample.pkl', directory='k_fold_validation/')

            # train a new model
            dm_train(input_file_path=os.path.join("inputs", "input_dfm.yaml"),
                     dataset_path=os.path.join("k_fold_validation", "name_pairs.txt"),
                     model_name="test00"+str(i))


            # generate vectors for queries (specified in dataset_path) 
            # using a model stored at pretrained_model_path and pretrained_vocab_path 
            dm_inference(os.path.join("inputs", "input_dfm.yaml"),
                         dataset_path=os.path.join("k_fold_validation", "queries.txt"), 
                         pretrained_model_path=os.path.join("models", "test00"+str(i), "test00"+str(i)+".model"), 
                         pretrained_vocab_path=os.path.join("models", "test00"+str(i), "test00"+str(i)+".vocab"),
                         inference_mode="vect",
                         scenario="queries/test")

            # generate vectors for candidates (specified in dataset_path) 
            # using a model stored at pretrained_model_path and pretrained_vocab_path 
            dm_inference(os.path.join("inputs", "input_dfm.yaml"),
                         dataset_path=os.path.join("k_fold_validation", "candidates.txt"), 
                         pretrained_model_path=os.path.join("models", "test00"+str(i), "test00"+str(i)+".model"), 
                         pretrained_vocab_path=os.path.join("models", "test00"+str(i), "test00"+str(i)+".vocab"),
                         inference_mode="vect",
                         scenario="candidates/test")

            # combine vectors stored in queries/test and save them in combined/queries_test
            combine_vecs(rnn_passes=['fwd', 'bwd'], 
                         input_scenario=os.path.join('queries', 'test'), 
                         output_scenario=os.path.join('combined', 'queries_test'), 
                         print_every=10)

            combine_vecs(rnn_passes=['fwd', 'bwd'], 
                 input_scenario=os.path.join('candidates', 'test'), 
                 output_scenario=os.path.join('combined', 'candidates_test'), 
                 print_every=10)

            # Select candidates based on L2-norm (aka faiss distance):
            # find candidates from candidate_scenario 
            # for queries specified in query_scenario
            candidates_pd = \
                candidate_ranker(query_scenario=os.path.join("combined", "queries_test"),
                                 candidate_scenario=os.path.join("combined", "candidates_test"), 
                                 ranking_metric="faiss", # two accepted value = ['cosine', faise]
                                 selection_threshold=.5, 
                                 num_candidates=3, 
                                 search_size=10, 
                                 verbose=False,
                                 use_predict=False,
                                 output_path=os.path.join("ranker_results", "test_candidates_deezymatch"), 
                                 pretrained_model_path=os.path.join("models", "test00"+str(i), "test00"+str(i)+".model"), 
                                 pretrained_vocab_path=os.path.join("models", "test00"+str(i), "test00"+str(i)+".vocab"))

            fuzzy_string_matching(os.path.join("ranker_results", "test_candidates_deezymatch.pkl"), os.path.join("k_fold_validation", "result.pkl"), directory='k_fold_validation/')

            total, retrieved, correct, r, p, f = compute_result(os.path.join("k_fold_validation", "result.pkl"))


            result = {
                'instance': total,
                'retrieved': retrieved, 
                'correct': correct, 
                'recall': r, 
                'precision': p, 
                'f-score': f
            }

            resultDict[i] = result
    
    finally:
        with open('k_fold_validation/resultDict_faiss_3.pickle', 'wb') as handle:
            pickle.dump(resultDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

In [4]:
if __name__ == '__main__':
    k_fold_validation()

2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] read input file: inputs/input_dfm.yaml
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] GPU was requested but not available.
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] pytorch will use: cpu
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] read CSV file: k_fold_validation/name_pairs.txt
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] number of labels, True: 24390 and False: 24390
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] Splitting the Dataset
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] finish splitting the Dataset. User time: 0.026088953018188477
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] splits are as follow:
train    34146
test      7318
val       7316
Name: split, dtype: int64
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] start creating a lookup table and convert characters to indices
2023-06-14 22:46:12 Sarahs-MacBook-Pro [INFO] -- create vocabulary
2023-06-14 22:46:15 Sarahs-MacBook-Pro [INFO] -- convert tokens to indices
2023-06-14




2023-06-14 22:46:16 Sarahs-MacBook-Pro [INFO] ******************************
2023-06-14 22:46:16 Sarahs-MacBook-Pro [INFO] **** (Bi-directional) GRU ****
2023-06-14 22:46:16 Sarahs-MacBook-Pro [INFO] ******************************
2023-06-14 22:46:16 Sarahs-MacBook-Pro [INFO] read inputs
2023-06-14 22:46:16 Sarahs-MacBook-Pro [INFO] create a two_parallel_rnns model
2023-06-14 22:46:16 Sarahs-MacBook-Pro [INFO] start fitting parameters
2023-06-14 22:46:16 Sarahs-MacBook-Pro [INFO] Number of batches: 3415
2023-06-14 22:46:16 Sarahs-MacBook-Pro [INFO] Number of epochs: 5


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3415 [00:00<?, ?it/s]




Total number of params: 3079263

two_parallel_rnns (
  (emb): Embedding(48409, 60), weights=((48409, 60),), parameters=2904540
  (rnn_1): GRU(60, 60, num_layers=2, dropout=0.01, bidirectional=True), weights=((180, 60), (180, 60), (180,), (180,), (180, 60), (180, 60), (180,), (180,), (180, 120), (180, 60), (180,), (180,), (180, 120), (180, 60), (180,), (180,)), parameters=109440
  (attn_step1): Linear(in_features=120, out_features=60, bias=True), weights=((60, 120), (60,)), parameters=7260
  (attn_step2): Linear(in_features=60, out_features=1, bias=True), weights=((1, 60), (1,)), parameters=61
  (fc1): Linear(in_features=480, out_features=120, bias=True), weights=((120, 480), (120,)), parameters=57720
  (fc2): Linear(in_features=120, out_features=2, bias=True), weights=((2, 120), (2,)), parameters=242
)


2023-06-14 22:55:47 Sarahs-MacBook-Pro [INFO] 06/14/2023_22:55:47 -- Epoch: 1/5; Train; loss: 0.220; acc: 0.904; precision: 0.889, recall: 0.924, macrof1: 0.904, weightedf1: 0.904


...

2023-06-15 01:55:03 Sarahs-MacBook-Pro [INFO] save test-data-class: /Users/sarah_shoilee/PycharmProjects/entity_linking/candidates/test/dataframe.df


  0%|          | 0/228 [00:00<?, ?it/s]

--- 2.7814338207244873 seconds ---
2023-06-15 01:55:06 Sarahs-MacBook-Pro [INFO] read input file: queries/test/input_dfm.yaml
2023-06-15 01:55:06 Sarahs-MacBook-Pro [INFO] GPU was requested but not available.
2023-06-15 01:55:06 Sarahs-MacBook-Pro [INFO] pytorch will use: cpu


-- Combine vectors
Reading vectors from queries/test/embeddings/rnn_fwd*
0000000 queries/test/embeddings/rnn_fwd_0
0000010 queries/test/embeddings/rnn_fwd_10
0000020 queries/test/embeddings/rnn_fwd_20
0000030 queries/test/embeddings/rnn_fwd_30
0000040 queries/test/embeddings/rnn_fwd_40
0000050 queries/test/embeddings/rnn_fwd_50
0000060 queries/test/embeddings/rnn_fwd_60
0000070 queries/test/embeddings/rnn_fwd_70
0000080 queries/test/embeddings/rnn_fwd_80
0000090 queries/test/embeddings/rnn_fwd_90
0000100 queries/test/embeddings/rnn_fwd_100
0000110 queries/test/embeddings/rnn_fwd_110
0000120 queries/test/embeddings/rnn_fwd_120


-- Combine IDs

0000000 queries/test/embeddings/rnn_indxs_0
0000010 queries/test/embe

1236it [15:43,  1.31it/s]

Total query: 1236 
Correct correspondence count: 856
Recall: 0.6925566343042071


Total query: 1236
Total retrieved: 1140
Correct correspondence count: 856 
Precision: 0.7508771929824561


F-measure: 0.7205387205387206



In [7]:
k = 3

with open('k_fold_validation/resultDict_faiss_3.pickle', 'rb') as handle:
    resultDict = pickle.load(handle)
    
    total_instance = sum([resultDict[i]['instance'] for i in range(0,k)])
    total_retrieved = sum([resultDict[i]['retrieved'] for i in range(0,k)])
    total_correct = sum([resultDict[i]['correct'] for i in range(0,k)])
    total_recall = sum([resultDict[i]['recall'] for i in range(0,k)])
    total_precision = sum([resultDict[i]['precision'] for i in range(0,k)])
    total_f_score = sum([resultDict[i]['f-score'] for i in range(0,k)])
    
    print(f"{total_instance/k} & {total_retrieved/k} & {total_correct/k} & {total_recall/k} & {total_precision/k} & {total_f_score/k}")

1236.0 & 833.6666666666666 & 656.6666666666666 & 0.5312837108953613 & 0.7957508719917016 & 0.6275777197097023


In [6]:
with open('k_fold_validation/resultDict_faiss_3.pickle', 'rb') as handle:
    resultDict = pickle.load(handle)
    
print(resultDict)

{0: {'instance': 1236, 'retrieved': 698, 'correct': 580, 'recall': 0.4692556634304207, 'precision': 0.830945558739255, 'f-score': 0.5997931747673216}, 1: {'instance': 1236, 'retrieved': 663, 'correct': 534, 'recall': 0.4320388349514563, 'precision': 0.8054298642533937, 'f-score': 0.5624012638230648}, 2: {'instance': 1236, 'retrieved': 1140, 'correct': 856, 'recall': 0.6925566343042071, 'precision': 0.7508771929824561, 'f-score': 0.7205387205387206}, 3: {}, 4: {}}


In [ ]:
{0: {'instance': 1236, 'retrieved': 698, 'correct': 580, 'recall': 0.4692556634304207, 'precision': 0.830945558739255, 'f-score': 0.5997931747673216}, 1: {'instance': 1236, 'retrieved': 663, 'correct': 534, 'recall': 0.4320388349514563, 'precision': 0.8054298642533937, 'f-score': 0.5624012638230648}, 2: {'instance': 1236, 'retrieved': 1140, 'correct': 856, 'recall': 0.6925566343042071, 'precision': 0.7508771929824561, 'f-score': 0.7205387205387206}, 3: {}, 4: {}}